In [124]:
import pandas as pd
import pickle
import os
import sys

In [119]:
df = pd.read_csv('/Users/shashankkumar/Documents/GitHub/MacroEcon/scratch/household and race.csv')
df.columns = df.iloc[0]
df = df[1:]
df = df.drop(df.columns[0],axis=1)
df['Tract Code'] =(df['Tract Code'].astype(float) * 100).astype(int)

In [135]:
df_eth = df[[               
                                    'Tract Code',
                          'Non-Hisp White Population',
                       'American Indian Pop- ulation',
       'Asian/ Hawaiian/ Pacific Islander Population',
                                 'Black Pop- ulation',
                                'Hispanic Population',
                'Other Population/ Two or More Races']]
df_eth = df_eth.rename(columns={
    'Tract Code': 'area',
    'Non-Hisp White Population': 'White',
    'American Indian Pop- ulation': 'AmericanIndian',
    'Asian/ Hawaiian/ Pacific Islander Population': 'AsianHawaiianPacificIslander',
    'Black Pop- ulation': 'Black',
    'Hispanic Population': 'Hispanic',
    'Other Population/ Two or More Races': 'Other'
})
df_melted = df_eth.melt(id_vars='area', 
                    value_vars=['White', 'AmericanIndian', 'AsianHawaiianPacificIslander', 'Black', 'Hispanic', 'Other'], 
                    var_name='ethnicity', 
                    value_name='count')

# Rename the ethnicity categories
ethnicity_mapping = {
    'White': 'European',
    'AmericanIndian': 'Maori',
    'AsianHawaiianPacificIslander': 'Pacific',
    'Black': 'Asian',
    'Hispanic': 'MELAA',
    'Other': 'Other'
}
df_melted['ethnicity'] = df_melted['ethnicity'].map(ethnicity_mapping)

# Multiply the area by 1000 to match the desired format
# df_melted['area'] = df_melted['area'] * 1000
df_eth = df_melted
df_eth = df_eth.sort_values(by='area')
df_eth['count'] = df_eth['count'].astype(int)
df_eth

,area,ethnicity,count
0,10101,European,963
976,10101,MELAA,197
732,10101,Asian,69
488,10101,Pacific,671
244,10101,Maori,3
...,...,...,...
975,990200,Asian,0
487,990200,Maori,0
243,990200,European,0
1219,990200,MELAA,0


In [134]:
df_eth['count']

0       963
976     197
732      69
488     671
244       3
       ... 
975       0
487       0
243       0
1219      0
1463      0
Name: count, Length: 1464, dtype: int64

In [122]:
df_house = df[['Tract Code', 
'Tract Population', 
'Number of Families', 
'# of House- holds']]
df_house = df_house.rename(columns={
    'Tract Code': 'area',
    'Tract Population': 'Population',
    'Number of Families': 'Families',
    '# of House- holds': 'Households'
})
df_house

,area,Population,Families,Households
1,10101,2026,344,1233
2,10102,2000,530,909
3,10201,2468,479,1321
4,10202,2074,451,1064
5,10300,4062,966,2151
...,...,...,...,...
240,980501,125,30,122
241,980600,1340,177,308
242,980900,1248,47,129
243,990100,0,0,0


Age

In [142]:
area = df['Tract Code']
population_tract = df['Tract Population']
df_age = pd.read_csv('/Users/shashankkumar/Documents/GitHub/MacroEcon/scratch/sf_age.csv')
df_age  = df_age[['Label (Grouping)', 'San Francisco city, California!!Total!!Estimate',
       'San Francisco city, California!!Male!!Estimate',
       'San Francisco city, California!!Female!!Estimate']
       ]
df_age = df_age.iloc[:-22].reset_index(drop=True)
df_age = df_age.drop(0)
df_age = df_age.drop(1)
df_age = df_age.rename(columns={
    'Label (Grouping)': 'age',
    'San Francisco city, California!!Total!!Estimate': 'Total',
    'San Francisco city, California!!Male!!Estimate': 'Male',
    'San Francisco city, California!!Female!!Estimate': 'Female',
    'Grouped_Labe': 'Grouped_Label'
})
df_age.reset_index(drop=True)
df_age['Total'] = df_age['Total'].str.replace(',', '').astype(int)
df_age['Male'] = df_age['Male'].str.replace(',', '').astype(int)
df_age['Female'] = df_age['Female'].str.replace(',', '').astype(int)
df_age['age'] = [
    'U19',
    'U19',
    'U19',
    'U19',
    '20t29',
    '20t29',
    '30t39',
    '30t39',
    '40t49',
    '40t49',
    '50t64',
    '50t64',
    '50t64',
    '65A',
    '65A',
    '65A',
    '65A',
    '65A',
]
df_age = df_age.groupby('age').sum().reset_index()
df_age['area'] = area.iloc[0]
df_age['population_count'] = df['Tract Population'].iloc[0]
df_repeated = pd.concat([df_age.assign(area=area_value,population_count = population_count_value) for area_value,population_count_value in zip(area,population_tract)], ignore_index=True)
df_age = df_repeated
df_melted = df_age.melt(id_vars=['area', 'age','population_count'], 
                    value_vars=['Male', 'Female'], 
                    var_name='gender', 
                    value_name='count')

# Convert the 'gender' column to lowercase
df_melted['gender'] = df_melted['gender'].str.lower()
df_age = df_melted
df_age = df_age.sort_values(by='area').reset_index(drop=True)
df_age['population_count'] = df_age['population_count'].astype(int)
df_age['population_count'] = df_age['population_count']/6
df_age['population_count'] = df_age['population_count'].astype(int)
df_age


,area,age,population_count,gender,count
0,10101,20t29,337,male,58274
1,10101,65A,337,female,80371
2,10101,U19,337,female,60920
3,10101,30t39,337,female,76854
4,10101,20t29,337,female,56299
...,...,...,...,...,...
2923,990200,40t49,0,male,59267
2924,990200,30t39,0,male,85786
2925,990200,20t29,0,male,58274
2926,990200,65A,0,female,80371


Save data

In [143]:
OUTPUT_DIR_SF = "/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/census_populations/SF/user_input"
SF_POP_OUT = os.path.join(OUTPUT_DIR_SF, "SF_POP.pkl")
att_dict = {'age_gender' : df_age,
    'ethnicity' : df_eth,}
with open(SF_POP_OUT, 'wb') as f:
    pickle.dump(att_dict, f)

Load Data gen by base_pop wrapper

In [144]:
df = pd.read_pickle("/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/census_populations/SF/synthetic_populations/base_population.pkl")

In [145]:
df

,area,age,gender,ethnicity
0,10101,20t29,male,European
1,10101,20t29,female,European
2,10101,20t29,female,European
3,10101,20t29,female,European
4,10101,20t29,female,European
...,...,...,...,...
858493,980900,40t49,female,European
858494,980900,40t49,male,MELAA
858495,980900,40t49,female,MELAA
858496,980900,40t49,male,Other
